In [1]:
# 1. Import Libraries
import torch
import itertools
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from tqdm import tqdm  # Progress bar

In [2]:
# 2. Setup perangkat (CPU/GPU)
print("CUDA Available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU Device Name:", torch.cuda.get_device_name(0))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device digunakan:", device)

CUDA Available: True
GPU Device Name: NVIDIA GeForce RTX 2070 SUPER
Device digunakan: cuda


In [3]:
# 3. Load Dataset
data = pd.read_csv(r'E:\Bot\MLD\winequality-white.csv', delimiter=';')  # Gunakan ';' sebagai pemisah
print("Dataset shape:", data.shape)
print("Columns:", data.columns)
print(data.head())

# Validasi dataset
if data.shape[1] < 2:
    raise ValueError("Dataset harus memiliki minimal satu kolom fitur dan satu kolom target!")

# Pisah fitur dan target
X = data.iloc[:, :-1].values  # Semua kolom kecuali terakhir
y = data.iloc[:, -1].values   # Kolom terakhir (quality)

# Pastikan label dimulai dari 0
y = y - y.min()
num_classes = len(np.unique(y))

Dataset shape: (4898, 12)
Columns: Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality'],
      dtype='object')
   fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0            7.0              0.27         0.36            20.7      0.045   
1            6.3              0.30         0.34             1.6      0.049   
2            8.1              0.28         0.40             6.9      0.050   
3            7.2              0.23         0.32             8.5      0.058   
4            7.2              0.23         0.32             8.5      0.058   

   free sulfur dioxide  total sulfur dioxide  density    pH  sulphates  \
0                 45.0                 170.0   1.0010  3.00       0.45   
1                 14.0                 132.0   0.9940  3.30       0.49   
2                 30.0                  97.0   0

In [4]:
# 4. Preprocessing
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Validasi data yang telah di-scale
if np.isnan(X_scaled).any() or np.isinf(X_scaled).any():
    raise ValueError("Dataset contains NaN or Inf values after scaling.")

# Bagi data menjadi training dan testing
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

# Convert ke PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Dataset dan DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# Hyperparameters
batch_sizes = [16, 32, 64, 128, 256, 512]
hidden_layers = [1, 2, 3]
neurons = [4, 8, 16, 32, 64]
activations = {
    "relu": nn.ReLU(),
    "sigmoid": nn.Sigmoid(),
    "tanh": nn.Tanh()
}
learning_rates = [10, 1, 0.1, 0.01, 0.001, 0.0001]
epochs = [1, 10, 25, 50, 100, 250]

In [5]:
# 5. Define MLP Model
class MLP(nn.Module):
    def __init__(self, input_size, hidden_layers, neurons, activation):
        super(MLP, self).__init__()
        layers = []
        for i in range(hidden_layers):
            if i == 0:
                layers.append(nn.Linear(input_size, neurons))
            else:
                layers.append(nn.Linear(neurons, neurons))
            layers.append(activation)
        layers.append(nn.Linear(neurons, num_classes))  # Output layer tanpa Softmax
        self.model = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.model(x)

In [6]:
# 6. Training and Evaluation Function
def train_and_evaluate(batch_size, hidden_layer, neuron, activation, lr, epoch):
    # DataLoader
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    # Model
    model = MLP(input_size=X_train.shape[1], hidden_layers=hidden_layer, neurons=neuron, activation=activations[activation])
    model.to(device)
    
    # Loss and Optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    # Training
    model.train()
    for ep in range(epoch):
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            
            # Forward pass
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            
            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
    
    # Evaluation
    model.eval()
    y_pred = []
    with torch.no_grad():
        for X_batch, _ in test_loader:
            X_batch = X_batch.to(device)
            outputs = model(X_batch)
            y_pred.extend(torch.argmax(outputs, axis=1).cpu().numpy())
    
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

In [ ]:
# 7. Hyperparameter Search
results = []
best_accuracy = 0
best_model = None

for params in tqdm(itertools.product(batch_sizes, hidden_layers, neurons, activations.keys(), learning_rates, epochs)):
    batch_size, hidden_layer, neuron, activation, lr, epoch = params
    print(f"\nRunning: Batch={batch_size}, Layers={hidden_layer}, Neurons={neuron}, Activation={activation}, LR={lr}, Epoch={epoch}")
    acc = train_and_evaluate(batch_size, hidden_layer, neuron, activation, lr, epoch)
    results.append((batch_size, hidden_layer, neuron, activation, lr, epoch, acc))
    print(f"Accuracy: {acc:.4f}")
    
    # Save best model
    if acc > best_accuracy:
        best_accuracy = acc
        best_model = (batch_size, hidden_layer, neuron, activation, lr, epoch)

# Save Results
results_df = pd.DataFrame(results, columns=["Batch Size", "Hidden Layers", "Neurons", "Activation", "Learning Rate", "Epochs", "Accuracy"])
results_df.to_csv("mlp_results.csv", index=False)
print("Results saved to mlp_results.csv")

print("\nBest Model Configuration:")
print(f"Batch Size={best_model[0]}, Hidden Layers={best_model[1]}, Neurons={best_model[2]}, Activation={best_model[3]}, Learning Rate={best_model[4]}, Epochs={best_model[5]}, Accuracy={best_accuracy:.4f}")

0it [00:00, ?it/s]


Running: Batch=16, Layers=1, Neurons=4, Activation=relu, LR=10, Epoch=1


1it [00:02,  2.45s/it]

Accuracy: 0.4480

Running: Batch=16, Layers=1, Neurons=4, Activation=relu, LR=10, Epoch=10


2it [00:05,  2.69s/it]

...

Accuracy: 0.5327

Running: Batch=64, Layers=2, Neurons=8, Activation=tanh, LR=0.01, Epoch=25


3981it [18:02:54,  4.86s/it]

Accuracy: 0.5418

Running: Batch=64, Layers=2, Neurons=8, Activation=tanh, LR=0.01, Epoch=50


3982it [18:02:59,  4.82s/it]

Accuracy: 0.5316

Running: Batch=64, Layers=2, Neurons=8, Activation=tanh, LR=0.01, Epoch=100
